In [ ]:
import numpy as np
import sys
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
import time
sys.path.append('../Functions/py_functions/') # This path is so that within each function file, you can import the other function files with relative paths
sys.path.append('../') # This path is so that we can import the functions folder from the root directory compared to where this file is
from Functions.py_functions.tire_model.tire_model_utils import *
from Functions.py_functions.constants import *
from Functions.py_functions.car_configuration import Car, to_vel_frame, to_car_frame, clip

In [ ]:
# generate a car object
car = Car()
car.set_tire(H_R20_18X6_7)
car.toe_front = 0
car.toe_rear = 0
car.update_car()

In [ ]:
# time the two versions of the s_r function
lenz = 1000
fx_targ = np.linspace(-3000, 3000, lenz)
sa = np.deg2rad(15)
fz = 2000
ia = np.deg2rad(-1.25)
flip = True
k0, bam0, fx0 = np.zeros(lenz), np.zeros(lenz), np.zeros(lenz)
t0 = time.time()
for ind, fx in enumerate(fx_targ):
    k0[ind], bam0[ind], fx0[ind] = car.s_r(fz, sa, 11.1, fx, i_a=ia, flip_s_a=flip)
print(time.time() - t0)
k1, bam1, fx1 = np.zeros(lenz), np.zeros(lenz), np.zeros(lenz)
t1 = time.time()
for ind, fx in enumerate(fx_targ):
    k1[ind], bam1[ind], fx1[ind] = car.fast_mf.s_r(fz, sa, 0.2, -0.3, 0.2, -0.3, 0, 0, 0, 82500.0, ia, 11.1, 0.0, 0.0, 1.0, flip, False, fx, 0)
print(time.time() - t1)

# plot the results
fig = make_subplots(rows=3, cols=1)
fig.add_trace(go.Scatter(x=fx_targ, y=k0, name='k0'), row=1, col=1)
fig.add_trace(go.Scatter(x=fx_targ, y=k1, name='k1'), row=1, col=1)
fig.add_trace(go.Scatter(x=fx_targ, y=fx0, name='fx0'), row=2, col=1)
fig.add_trace(go.Scatter(x=fx_targ, y=fx1, name='fx1'), row=2, col=1)
fig.add_trace(go.Scatter(x=fx_targ, y=bam0, name='bam0'), row=3, col=1)
fig.add_trace(go.Scatter(x=fx_targ, y=bam1, name='bam1'), row=3, col=1)
fig.show()
# plot kappa vs fx
fig = px.scatter(x=k0, y=fx0)
fig.add_scatter(x=k1, y=fx1, mode='markers')
fig.show()

In [ ]:
kappas = np.linspace(-0.3, 0.2, 500)
fz, sa, v, _, ia = 15295, 0.008726646259971648, 10.0, -794.0029381399711, -0.02181661564992912
fx, _, _ = car.fast_mf.solve_sr_sweep(fz, sa, kappas, 82700, ia, v, 0.0, 0.0, 0.65, False)
fig = px.scatter(x=kappas, y=fx)
fig.show()

In [ ]:
fzs = np.linspace(1, 3000, 500)
sa, v, _, ia = 0.0, 10.0, -794.0029381399711, 0.0
fxx = np.zeros(len(fzs))
for ind, fz in enumerate(fzs):
    fx, _, _ = car.fast_mf.solve_sr_sweep(fz, sa, np.array([0.0]), 82700, ia, v, 0.0, 0.0, 0.65, False)
    fxx[ind] = fx[0]
fig = px.scatter(x=fzs, y=fxx)
fig.show()

In [ ]:
dfz = (fzs - car.mf_tire.FNOMIN) / car.mf_tire.FNOMIN
svx = (car.mf_tire.PVX1 + car.mf_tire.PVX2 * dfz) * fzs
shx = car.mf_tire.PHX1 + car.mf_tire.PHX2 * dfz
kx = (car.mf_tire.PKX1 + car.mf_tire.PKX2 * dfz) * np.exp(car.mf_tire.PKX3 * dfz)
fig = px.scatter(x=fzs, y=dfz)
fig.show()
fig = px.scatter(x=fzs, y=svx)
fig.show()
fig = px.scatter(x=fzs, y=shx)
fig.show()
fig = px.scatter(x=fzs, y=kx)
fig.add_scatter(x=fzs, y=(car.mf_tire.PKX1 + car.mf_tire.PKX2 * dfz))
fig.show()
print(car.mf_tire.PKX1)
print(car.mf_tire.PKX2)
print(car.mf_tire.PKX3)